In [ ]:
### Inputs: traces_dict, node_details_dict and trace_details_dict
# Node details dict= nid: [nis, type]
### Config file: DB split and SLtype split
### Outputs: updated_node_details

In [71]:
import pickle
import yaml
import random
import json

import networkx as nx
import numpy as np

In [2]:
def pkl_to_dict(file_path):
    with open(file_path, 'rb') as pkl_file:
        T_prime = pickle.load(pkl_file)
    return T_prime

def read_yaml(file):
    with open(file, 'r') as f:
        data = yaml.safe_load(f)
    return data

def build_digraph_from_tracesdict(traces_dict):

    full_graph_edge_list = []
    for edge_list in traces_dict.values():
        full_graph_edge_list.extend(edge_list)
    G = nx.DiGraph()
    G.add_edges_from(full_graph_edge_list)

    return G


In [3]:
# Read in configs
config = read_yaml('enrichment_config.yaml')
databases = config['Databases']
sl_types = config['SLTypeSplit']

db_split_arr = [[db_name, info['percentage']] for db_name, info in databases.items()]# [[DB1, 30%],...]
sl_type_split = [[sl_type, info['percentage']] for sl_type, info in sl_types.items()]# [[Relay, 30%],...]

# Node details dict= nid: [nis, SF, DB_name] (or) [nis, SL, SL_type]
node_dets = pkl_to_dict('node_details_data.pkl')
sf_arr = [nid for nid, n_info in node_dets.items() if n_info[1] == "db"]
sl_arr = [nid for nid, n_info in node_dets.items() if n_info[1] != "db"]

sf_count = len(sf_arr)
sl_count = len(sl_arr)
total_nodes = sf_count + sl_count

def percent_to_count(arr, count):
    for idx, i in enumerate(arr):
        name = i[0]
        arr[idx] = [name,int(count * (i[1])/100)]
    return arr

db_split_arr = percent_to_count(db_split_arr, sf_count)
sl_type_split = percent_to_count(sl_type_split, sl_count)

print(len(sf_arr))
print(len(sl_arr))
print(db_split_arr)
print(sl_type_split)

def assign_nodes_to_types(arr, sfsl_arr):
    # Assign nodes to db and sl types
    for i in arr:
        name = i[0]
        for _ in range(i[1]):
            nid = sfsl_arr.pop(random.randint(0, len(sfsl_arr) - 1))
            node_dets[nid].append(name)
    return node_dets
node_dets = assign_nodes_to_types(db_split_arr, sf_arr)
node_dets = assign_nodes_to_types(sl_type_split, sl_arr)

2330
1652
[['MongoDB', 699], ['Redis', 699], ['Postgres', 932]]
[['Relay', 495], ['High', 330], ['Low', 826]]


In [4]:
check = []
for i in node_dets.values():
    if len(i) != 3:
        check.append(i)
print(len(check))
print(len(node_dets))

1
3982


In [96]:
'''
Object id Enrichment
'''

class Wl_config:
    def __init__(self, record_count, record_size_dist,\
                 data_access_pattern, rw_ratio, async_sync_ratio, seed):
        self.record_count = record_count
        self.record_size_dist = record_size_dist
        self.data_access_pattern = data_access_pattern
        self.rw_ratio = rw_ratio
        self.async_sync_ratio = async_sync_ratio
        self.seed = seed

        # Setting seed
        np.random.seed(self.seed)
        random.seed(self.seed)
        # Generate object sizes and data access pattern
        self.obj_ids_list = np.arange(1, self.record_count + 1)
        self.object_sizes_dict = self.generate_object_sizes()
        self.probabilities = self.generate_data_access_pattern()

    def generate_object_sizes(self):
        if self.record_size_dist == 'lognormal':
            obj_sizes = np.random.lognormal(mean=np.log(self.record_count), \
                                                 sigma=np.log(self.record_count), \
                                                 size=self.record_count)
        elif self.record_size_dist == 'uniform':
            obj_sizes = np.random.uniform(low=1, high=self.record_count, size=self.record_count)
        else:
            raise ValueError('Invalid record size distribution')
        return dict(zip(self.obj_ids_list, obj_sizes))
    
    def generate_data_access_pattern(self):
        if self.data_access_pattern == 'zipfian':
            alpha = 1.2
            probabilities = np.random.zipf(alpha, len(self.obj_ids_list))
            probabilities = probabilities / probabilities.sum()
        elif self.data_access_pattern == 'uniform':
            probabilities = np.ones(len(self.obj_ids_list)) / len(self.obj_ids_list)
        else:
            raise ValueError('Invalid data access pattern')
        return probabilities


def gen_sfnode_dataops(sf_node, G_agg, wl_config, node_dets):
    '''
    For a given sf node, generate indegree num of data ops
    Return: ops_dict= Key: op_id, Value: op_packet
    op_packet = {'op_id': op_id, 'op_type': op_type, 'op_obj_id': op_obj_id,\
                 'op_obj_size':op_obj_size,'db': sf_node_db}
    '''
    obj_ids_list = wl_config.obj_ids_list
    obj_sizes_dict = wl_config.object_sizes_dict
    data_acc_probabilities = wl_config.probabilities
    w_prob = wl_config.rw_ratio / (1 + wl_config.rw_ratio)
    obj_count = wl_config.record_count

    sf_node_db = node_dets[sf_node][2]
    total_ops = G_agg.in_degree(sf_node)# gen indeg num of data ops

    ops_dict = {}   # key: op_id, value: op_packet
    for op_id in range(1, total_ops + 1):
        # print("opid", op_id)
        op_type = 'write' if random.random() < w_prob else 'read'
        op_obj_id = np.random.choice(obj_ids_list,\
                                     p=data_acc_probabilities)# Select by data access pattern
        # print("op_obj_id", op_obj_id)
        op_obj_size = obj_sizes_dict[op_obj_id]
        operation = {'op_id': op_id, 'op_type': op_type, 'op_obj_id': int(op_obj_id),\
                      'op_obj_size':int(op_obj_size),'db': sf_node_db} # op_packet
        ops_dict[op_id] = operation
    
    return ops_dict

# convert edges_list to node_calls_dict format
def convert_to_node_calls_dict(edges_list):
    node_calls_dict = {}
    for edge in edges_list:
        if edge[0] not in node_calls_dict:
            node_calls_dict[edge[0]] = []
        node_calls_dict[edge[0]].append([edge[1], -1]) # [dm node, op_id] (-1 for SL)
    return node_calls_dict

wl1 = Wl_config(100, 'uniform', 'zipfian', 0.5, 0.5, 50) # to be read from config file

traces_dict = pkl_to_dict('traces/500nodes_4500traces.pkl')
G_agg = build_digraph_from_tracesdict(traces_dict)

# Generate data ops for each sf node
overall_data_ops = {}   # key: sf_node, value: ops_dict
check = []
for node in node_dets:
    if node in G_agg.nodes() and node_dets[node][1] == 'db':
        overall_data_ops[node] = gen_sfnode_dataops(node, G_agg, wl1, node_dets)

In [87]:
print(len(overall_data_ops))

56


In [97]:
'''
Making the trace packet:
trace_packet = [t_node_calls_dict, t_data_ops_dict]
'''
def get_pop_first_dict_item(d):
    first_key = list(d.keys())[0]
    first_item = d.pop(first_key)
    return first_key, first_item

ctr = 0
err_ctr = 0
all_trace_packets = {}
for tid in traces_dict:
    t_node_calls_dict = convert_to_node_calls_dict(traces_dict[tid])
    t_data_ops_dict = {} # key: data op id, value: data op packet
    for t_node in t_node_calls_dict:
        for idx, dm_node in enumerate(t_node_calls_dict[t_node]):# why is it not entering the if loop?
            
            if dm_node[0] in overall_data_ops: # ie if dm node is a sf node
                
                # select a data op id from the data ops dict and pop it
                if len(overall_data_ops[dm_node[0]]) == 0:
                    print("Error: No data ops for sf node", dm_node[0])
                    err_ctr += 1
                    break
                ctr += 1
                op_id, op_packet = get_pop_first_dict_item(overall_data_ops[dm_node[0]])
                
                t_node_calls_dict[t_node][idx][1] = op_id
                t_data_ops_dict[op_id] = op_packet
    trace_packet = [t_node_calls_dict, t_data_ops_dict]
    all_trace_packets[tid] = trace_packet

# CHECK IF INDEG IS EQUAL TO NUM OF DM CALLS



Error: No data ops for sf node 845d0363e92434814e1b772015c90c08217a4fe7446b06dbdcf0c62c69963861
Error: No data ops for sf node 960f5e6d61e96626a514d7a3fed6d8e81ffebd7d0051b3735d8d342261662e26
Error: No data ops for sf node 95a9c24648c4754da55afdd47e854bff99381b430f915769c6370ba196af2fd0
Error: No data ops for sf node cc10726ed901dca4bf7f553bc97e1223103f20c3df5bed167bbc794824540870
Error: No data ops for sf node a7f9f6e2f660b7646118d88512aea0384de690ebeeda64cedc24c72ba2f6d8f4
Error: No data ops for sf node 845d0363e92434814e1b772015c90c08217a4fe7446b06dbdcf0c62c69963861
Error: No data ops for sf node a18570bba9ced2ba23026e03bff4f220b336731022c1c399c89ac47d566dce1d
Error: No data ops for sf node d1b569850f04df209add2bbbe4c8f7b93732b0b5bcf0a4d7b13770e3317fd3d3
Error: No data ops for sf node d1b569850f04df209add2bbbe4c8f7b93732b0b5bcf0a4d7b13770e3317fd3d3
Error: No data ops for sf node d1b569850f04df209add2bbbe4c8f7b93732b0b5bcf0a4d7b13770e3317fd3d3
Error: No data ops for sf node 1ed1099e4

In [98]:
print(ctr)
print(err_ctr)

60
118
